# Browns Tracking Analysis Template

This scaffold is aligned to the Performance Science work project brief and focuses on:
1) speed-band workload metrics, 2) rolling peak-demand windows,
3) transparent session segmentation, and 4) slide-ready visuals.

## 0. Setup
- Update thresholds/config values as needed for your final narrative.
- Keep outputs reproducible by saving figures/tables to `outputs/`.

## Unit Assumptions
- Speed conversion: yards/second -> mph (x 2.0454545).
- Acceleration conversion: yards/second^2 -> m/second^2 (x 0.9144).

In [ ]:
import pandas as pd
from IPython.display import display

from browns_tracking.metrics import (
    compute_peak_demand_timeseries,
    peak_distance_table,
    relative_speed_bands,
    session_extrema_table,
    summarize_speed_bands,
    top_non_overlapping_windows,
)
from browns_tracking.config import default_project_paths
from browns_tracking.pipeline import (
    compute_data_quality_summary,
    compute_session_event_counts,
    load_tracking_data,
    split_early_late_summary,
    summarize_session,
)
from browns_tracking.presets import preferred_performance_model
from browns_tracking.segmentation import (
    build_coach_phase_summary,
    detect_segments,
    summarize_segments,
)
from browns_tracking.visuals import (
    plot_intensity_timeline,
    plot_movement_map,
    plot_peak_demand_summary,
    save_figure,
)

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 200)

In [ ]:
paths = default_project_paths()
DATA_PATH = paths.data_file
OUTPUT_DIR = paths.output_dir
FIG_DIR = OUTPUT_DIR / 'figures'
TABLE_DIR = OUTPUT_DIR / 'tables'
FIG_DIR.mkdir(parents=True, exist_ok=True)
TABLE_DIR.mkdir(parents=True, exist_ok=True)
model = preferred_performance_model()

model.name, DATA_PATH

## Preferred Threshold Model

In [ ]:
model_summary = {
    'model_name': model.name,
    'rationale': model.rationale,
    'hsr_threshold_mph': model.peak_demand_config.hsr_threshold_mph,
    'accel_threshold_ms2': model.peak_demand_config.accel_threshold_ms2,
    'decel_threshold_ms2': model.peak_demand_config.decel_threshold_ms2,
    'rest_speed_threshold_mph': model.segmentation_config.rest_speed_threshold_mph,
    'rest_min_duration_s': model.segmentation_config.rest_min_duration_s,
    'speed_bands': [
        f"{b.name}: {b.lower_mph}-{b.upper_mph if b.upper_mph is not None else 'max'} mph"
        for b in model.absolute_speed_bands
    ],
}
pd.Series(model_summary, name='value').to_frame()

## Definitions and Threshold Rationale

In [ ]:
definitions = pd.DataFrame(
    [
        {'definition': 'Speed bands (mph)', 'value': ', '.join([f"{b.name} {b.lower_mph}-{b.upper_mph if b.upper_mph is not None else 'max'}" for b in model.absolute_speed_bands])},
        {'definition': 'HSR threshold (mph)', 'value': f"{model.peak_demand_config.hsr_threshold_mph:.1f}"},
        {'definition': 'Sprint threshold (mph)', 'value': '16.0'},
        {'definition': 'Accel/Decel thresholds (m/s^2)', 'value': f">= {model.peak_demand_config.accel_threshold_ms2:.1f} / <= {model.peak_demand_config.decel_threshold_ms2:.1f}"},
        {'definition': 'Event definition', 'value': 'Contiguous threshold exposure >= 1.0 s'},
        {'definition': 'Relative bands', 'value': 'Anchored to session max speed'},
        {'definition': 'Rationale', 'value': 'Thresholds reflect common team-practice reporting and coach readability'},
    ]
)
display(definitions)

## 1. Load Data and Session QA Summary

In [ ]:
df = load_tracking_data(DATA_PATH)
qa_summary = compute_data_quality_summary(df)
session_summary = pd.Series(summarize_session(df), name='value').to_frame()
display(pd.DataFrame([qa_summary]))
display(session_summary)
pd.DataFrame([qa_summary]).to_csv(TABLE_DIR / 'data_quality_summary.csv', index=False)

df.head()

## 2. Speed Bands (Absolute + Relative-to-Max)

In [ ]:
absolute_bands = list(model.absolute_speed_bands)
absolute_band_summary = summarize_speed_bands(df, absolute_bands)

relative_bands = relative_speed_bands(
    df['speed_mph'].max(),
    percent_edges=model.relative_band_edges,
)
relative_band_summary = summarize_speed_bands(df, relative_bands)

display(absolute_band_summary)
display(relative_band_summary)

absolute_band_summary.to_csv(TABLE_DIR / 'absolute_speed_band_summary.csv', index=False)
relative_band_summary.to_csv(TABLE_DIR / 'relative_speed_band_summary.csv', index=False)

## 3. Rolling Peak-Demand Windows + Event Counts

In [ ]:
peak_cfg = model.peak_demand_config

rolling = compute_peak_demand_timeseries(df, peak_cfg)
distance_table = peak_distance_table(rolling, peak_cfg.distance_windows_s)
top_1m_distance_windows = top_non_overlapping_windows(
    rolling, metric_column='distance_60s_yd', window_s=60, top_n=3
)
extrema_table = session_extrema_table(df)
event_counts = compute_session_event_counts(
    df,
    hsr_threshold_mph=peak_cfg.hsr_threshold_mph,
    accel_threshold_ms2=peak_cfg.accel_threshold_ms2,
    decel_threshold_ms2=peak_cfg.decel_threshold_ms2,
)
early_late_summary = split_early_late_summary(
    df,
    hsr_threshold_mph=peak_cfg.hsr_threshold_mph,
    accel_threshold_ms2=peak_cfg.accel_threshold_ms2,
    decel_threshold_ms2=peak_cfg.decel_threshold_ms2,
)

display(distance_table)
display(top_1m_distance_windows)
display(extrema_table)
display(pd.DataFrame([event_counts]))
display(early_late_summary)

distance_table.to_csv(TABLE_DIR / 'peak_distance_windows.csv', index=False)
top_1m_distance_windows.to_csv(TABLE_DIR / 'top_1m_distance_windows.csv', index=False)
extrema_table.to_csv(TABLE_DIR / 'session_extrema.csv', index=False)
pd.DataFrame([event_counts]).to_csv(TABLE_DIR / 'session_event_counts.csv', index=False)
early_late_summary.to_csv(TABLE_DIR / 'early_vs_late_summary.csv', index=False)

## 4. Session Segmentation and Coach Phases (Merged for Context)

In [ ]:
segmented_df, segment_boundaries = detect_segments(df, model.segmentation_config)
segment_summary = summarize_segments(segmented_df, speed_bands=absolute_bands)
coach_df, coach_phase_summary = build_coach_phase_summary(
    segmented_df,
    min_phase_duration_s=30.0,
    max_phases=8,
    hsr_threshold_mph=peak_cfg.hsr_threshold_mph,
    accel_threshold_ms2=peak_cfg.accel_threshold_ms2,
    decel_threshold_ms2=peak_cfg.decel_threshold_ms2,
)

raw_segment_count = int(segment_boundaries.shape[0])
print(f'Raw algorithmic segments detected: {raw_segment_count}')
display(coach_phase_summary)

segment_boundaries.to_csv(TABLE_DIR / 'raw_segment_boundaries.csv', index=False)
segment_summary.to_csv(TABLE_DIR / 'raw_segment_summary.csv', index=False)
coach_phase_summary.to_csv(TABLE_DIR / 'coach_phase_summary.csv', index=False)

## 5. Visual Template 1: Movement Map (X-Y)

In [ ]:
fig1, _ = plot_movement_map(coach_df, segment_col='coach_phase_label', highlight_top_n=3)
save_figure(fig1, FIG_DIR / 'movement_map.png')
fig1

## 6. Visual Template 2: Intensity Timeline

In [ ]:
fig2, _ = plot_intensity_timeline(
    segmented_df,
    top_windows=top_1m_distance_windows,
    hsr_threshold_mph=peak_cfg.hsr_threshold_mph,
)
save_figure(fig2, FIG_DIR / 'intensity_timeline.png')
fig2

## 7. Visual Template 3: Peak-Demand Summary

In [ ]:
fig3, _ = plot_peak_demand_summary(distance_table, extrema_table)
save_figure(fig3, FIG_DIR / 'peak_demand_summary.png')
fig3

## 8. Slide Export Checklist
- `outputs/figures/movement_map.png`
- `outputs/figures/intensity_timeline.png`
- `outputs/figures/peak_demand_summary.png`
- `outputs/tables/coach_phase_summary.csv` for coach-context phases
- `outputs/tables/session_event_counts.csv` and `outputs/tables/early_vs_late_summary.csv`
- `outputs/tables/*.csv` supporting tables for notebook/slide text